### Creacion de sistema para crear CSV

In [110]:
# Paso 1: Importar librerías necesarias
import csv
import sqlite3
from datetime import datetime

In [112]:
import datetime

# Lista para almacenar registros únicos
registros = set()

# Función para validar y agregar registros
def agregar_registro(fecha, mesReporte, añoReporte, tipoRegistro, monto):
    # Validación de tipo de registro (Ingreso o Egreso)
    if tipoRegistro not in ["Ingreso", "Egreso"]:
        print("Error: El tipo de registro debe ser 'Ingreso' o 'Egreso'.")
        return False
    
    # Validación de formato de fecha
    try:
        fecha_obj = datetime.datetime.strptime(fecha, "%d-%m-%Y")
    except ValueError:
        print("Error: La fecha debe tener el formato 'DD-MM-YYYY'.")
        return False

    # Validación de formato numérico para mesReporte, añoReporte y monto
    try:
        mesReporte = int(mesReporte)
        añoReporte = int(añoReporte)
        monto = float(monto)
    except ValueError:
        print("Error: Datos inconsistentes. Mes, Año y Monto deben ser numéricos.")
        return False

    # Validación de mes dentro del rango 1-12
    if not (1 <= mesReporte <= 12):
        print("Error: El mes debe estar en el rango 1-12.")
        return False

    # Validación de que la fecha corresponde al mes y año de reporte
    if fecha_obj.month != mesReporte or fecha_obj.year != añoReporte:
        print("Error: La fecha no corresponde con el mes y año del reporte.")
        return False

    # Validación de monto positivo para 'Ingreso' o negativo/menor para 'Egreso'
    if tipoRegistro == "Ingreso" and monto <= 0:
        print("Error: El monto para 'Ingreso' debe ser positivo.")
        return False
    elif tipoRegistro == "Egreso" and monto >= 0:
        print("Error: El monto para 'Egreso' debe ser negativo.")
        return False

    # Crear una tupla con el registro para validar duplicados
    registro = (fecha, mesReporte, añoReporte, tipoRegistro, monto)
    if registro in registros:
        print("Error: Registro duplicado detectado ->", registro)
        return False

    # Agregar registro al conjunto si pasa todas las validaciones
    registros.add(registro)
    print("Registro agregado exitosamente:", registro)
    return True

### Cargar registros individualmente y verificar si hay registros duplicados.

In [114]:
# Paso 3: Agregar registros de prueba
# Error de tipo de registro
agregar_registro("8-11-2023", 11, 2023, "Retiro", 4500)
# Error formato Fecha
agregar_registro("2023-11-08", 11, 2023, "Ingreso", 4500)
# Error de valores
agregar_registro("8-11-2023", "Nov", 2023, "Ingreso", "cuatro mil quinientos")
#Error de mes fuero de rango
agregar_registro("8-13-2023", 13, 2023, "Ingreso", 4500)
#Error de inconsistencia entre la fecha y el mes/año del reporte
agregar_registro("8-11-2023", 10, 2023, "Ingreso", 4500)
# Error de monto negativo para "Ingreso"
agregar_registro("8-11-2023", 11, 2023, "Ingreso", -4500)

# Datos Correctos
agregar_registro("8-11-2023", 11, 2023, "Ingreso", 4500)
agregar_registro("9-11-2023", 11, 2023, "Egreso", -3000)


Error: El tipo de registro debe ser 'Ingreso' o 'Egreso'.
Error: La fecha debe tener el formato 'DD-MM-YYYY'.
Error: Datos inconsistentes. Mes, Año y Monto deben ser numéricos.
Error: La fecha debe tener el formato 'DD-MM-YYYY'.
Error: La fecha no corresponde con el mes y año del reporte.
Error: El monto para 'Ingreso' debe ser positivo.
Registro agregado exitosamente: ('8-11-2023', 11, 2023, 'Ingreso', 4500.0)
Registro agregado exitosamente: ('9-11-2023', 11, 2023, 'Egreso', -3000.0)


True

In [116]:
#Paso 3.1 con datos en carga masiva mediante un csv
# Paso 3.1 - Carga Masiva desde un Archivo CSV con Validación de Duplicados e Inconsistencias
def carga_masiva_desde_csv(nombre_archivo):
    with open(nombre_archivo, mode="r") as archivo_csv:
        reader = csv.reader(archivo_csv)
        next(reader)  # Saltar encabezado
        
        for fila in reader:
            # Verificar que la fila tenga todos los campos necesarios
            if len(fila) != 5:
                print(f"Error: Registro con formato incorrecto -> {fila}")
                continue
            
            fecha, mesReporte, añoReporte, tipoRegistro, monto = fila
            
            # Validar formato de datos
            try:
                mesReporte = int(mesReporte)
                añoReporte = int(añoReporte)
                monto = float(monto)
            except ValueError:
                print(f"Error: Datos inconsistentes en el registro -> {fila}")
                continue
            
            # Validar tipo de registro
            if tipoRegistro not in ["Ingreso", "Egreso"]:
                print(f"Error: Tipo de registro inválido en el registro -> {fila}")
                continue
            
            # Validar duplicados
            registro = (fecha, mesReporte, añoReporte, tipoRegistro, monto)
            if registro in registros:
                print(f"Registro duplicado detectado, omitiendo -> {registro}")
                continue
            
            # Agregar el registro si pasa todas las validaciones
            registros.add(registro)
            print("Registro agregado:", registro)

# Llamada a la función de carga masiva desde archivo CSV
carga_masiva_desde_csv("datos_masivos.csv")

Registro agregado: ('09-09-2023', 9, 2023, 'Ingreso', 4256.15)
Registro agregado: ('13-07-2023', 7, 2023, 'Egreso', -1908.92)
Registro agregado: ('01-10-2023', 10, 2023, 'Ingreso', 3039.84)
Registro agregado: ('29-12-2023', 12, 2023, 'Egreso', -2142.88)
Registro agregado: ('29-12-2023', 12, 2023, 'Ingreso', 1123.44)
Registro agregado: ('08-12-2023', 12, 2023, 'Egreso', -4760.95)
Registro agregado: ('04-11-2023', 11, 2023, 'Ingreso', 2056.87)
Registro agregado: ('29-09-2023', 9, 2023, 'Ingreso', 2168.37)
Registro agregado: ('29-12-2023', 12, 2023, 'Egreso', -4092.04)
Registro agregado: ('12-03-2023', 3, 2023, 'Ingreso', 1635.75)
Registro agregado: ('20-09-2023', 9, 2023, 'Egreso', -2536.18)
Registro agregado: ('14-02-2023', 2, 2023, 'Ingreso', 1884.52)
Registro agregado: ('25-02-2023', 2, 2023, 'Egreso', -3634.59)
Registro agregado: ('05-06-2023', 6, 2023, 'Egreso', -4992.52)
Registro agregado: ('01-10-2023', 10, 2023, 'Ingreso', 2252.82)
Registro agregado: ('04-11-2023', 11, 2023, 'Egr

In [118]:
# Paso 4: Guardar en CSV
def guardar_en_csv(nombre_archivo="reporte.csv"): 
    with open(nombre_archivo, mode="w", newline="") as archivo_csv:
        writer = csv.writer(archivo_csv)
        writer.writerow(["fecha", "mesReporte", "anioReporte", "tipoRegistro", "monto"])  # Encabezado
        for registro in registros:
            writer.writerow(registro)  # Escribir cada registro
    print(f"Archivo {nombre_archivo} guardado exitosamente.")

# Llamada para guardar en CSV
guardar_en_csv()


Archivo reporte.csv guardado exitosamente.


In [120]:
# Paso 5: Cargar el CSV en la base de datos SQL
def cargar_a_base_datos(nombre_archivo="reporte.csv", nombre_bd="respaldo.db"):
    conn = sqlite3.connect(nombre_bd)
    cursor = conn.cursor()

    # Crear tabla si no existe
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS flujo_efectivo (
            fecha TEXT,
            mesReporte INTEGER,
            añoReporte INTEGER,
            tipoRegistro TEXT,
            monto REAL
        )
    ''')

    # Leer el CSV e insertar datos
    with open(nombre_archivo, mode="r") as archivo_csv:
        reader = csv.reader(archivo_csv)
        next(reader)  # Saltar encabezado
        for row in reader:
            # Validar duplicados en la base de datos
            cursor.execute("SELECT * FROM flujo_efectivo WHERE fecha=? AND mesReporte=? AND añoReporte=? AND tipoRegistro=? AND monto=?", row)
            if cursor.fetchone():
                print("Registro duplicado en la base de datos, omitiendo:", row)
            else:
                cursor.execute("INSERT INTO flujo_efectivo (fecha, mesReporte, añoReporte, tipoRegistro, monto) VALUES (?, ?, ?, ?, ?)", row)

    conn.commit()
    conn.close()
    print("Datos cargados en la base de datos exitosamente.")

# Llamada para cargar datos en la base de datos
cargar_a_base_datos()


Registro duplicado en la base de datos, omitiendo: ['9-11-2023', '11', '2023', 'Egreso', '-3000.0']
Registro duplicado en la base de datos, omitiendo: ['8-11-2023', '11', '2023', 'Ingreso', '4500.0']
Datos cargados en la base de datos exitosamente.


In [122]:
# Paso 6: Mostrar registros y limpiar
def mostrar_registros(nombre_bd="respaldo.db"):
    conn = sqlite3.connect(nombre_bd)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM flujo_efectivo")
    registros = cursor.fetchall()
    conn.close()
    print("Registros en la base de datos:")
    for registro in registros:
        print(registro)

# Llamada para mostrar registros en la base de datos
mostrar_registros()


Registros en la base de datos:
('9-11-2023', 11, 2023, 'Egreso', -3000.0)
('8-11-2023', 11, 2023, 'Ingreso', 4500.0)
('25-02-2023', 2, 2023, 'Egreso', -3634.59)
('01-10-2023', 10, 2023, 'Ingreso', 2252.82)
('08-12-2023', 12, 2023, 'Egreso', -4760.95)
('09-09-2023', 9, 2023, 'Egreso', -2889.68)
('11-10-2023', 10, 2023, 'Ingreso', 2416.12)
('04-11-2023', 11, 2023, 'Egreso', -4186.78)
('13-08-2023', 8, 2023, 'Egreso', -2597.68)
('25-11-2023', 11, 2023, 'Egreso', -3193.72)
('20-09-2023', 9, 2023, 'Ingreso', 2160.96)
('29-11-2023', 11, 2023, 'Egreso', -4030.64)
('01-10-2023', 10, 2023, 'Ingreso', 3039.84)
('29-12-2023', 12, 2023, 'Egreso', -4092.04)
('29-12-2023', 12, 2023, 'Egreso', -2142.88)
('29-09-2023', 9, 2023, 'Ingreso', 2168.37)
('09-09-2023', 9, 2023, 'Ingreso', 4256.15)
('20-09-2023', 9, 2023, 'Egreso', -2536.18)
('14-02-2023', 2, 2023, 'Ingreso', 1884.52)
('05-06-2023', 6, 2023, 'Egreso', -4094.27)
('29-12-2023', 12, 2023, 'Ingreso', 1123.44)
('09-08-2023', 8, 2023, 'Egreso', -24